In [8]:
import os
import time
import copy
import random
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import IPython

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Start Connecting

In [9]:
import socket


server_address = ('localhost', 8080)

print("Trying to connect...")

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    try:
        # 서버에 연결 시도 
        s.connect(server_address)
        
        # 연결 성공 시 서버 정보 출력
        print(f"연결이 완료되었습니다. 서버 주소: {server_address[0]}, 포트: {server_address[1]}")
        print(f"로컬 주소: {s.getsockname()}")  # 클라이언트(로컬) 주소 출력
            
    except socket.error as e:
        # 연결 실패 시 오류 메시지 출력
        print(f"서버에 연결할 수 없습니다: {e}")



Trying to connect...
연결이 완료되었습니다. 서버 주소: localhost, 포트: 8080
로컬 주소: ('127.0.0.1', 14087)


# ResNet18 Model

In [4]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock_18(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock_18, self).__init__()
        self.conv_1 = conv3x3(inplanes, planes, stride)
        self.bn_1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv_2 = conv3x3(planes, planes)
        self.bn_2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv_1(x)
        out = self.bn_1(out)
        out = self.relu(out)

        out = self.conv_2(out)
        out = self.bn_2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out

class ResNet_18(nn.Module):
    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3

        super(ResNet_18, self).__init__()
        self.conv1g = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bng = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512*block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes*block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1g(x)
        x = self.bng(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x_full = x.view(x.size(0), -1)
        logits = self.fc(x_full)
        #probas = F.softmax(logits, dim=1)
        return logits, x_full

def resnet18(num_classes, GRAYSCALE):
    model = ResNet_18(block=BasicBlock_18, layers=[2,2,2,2], num_classes=num_classes, grayscale=GRAYSCALE)
    return model

# Client Model Training

In [5]:
def avg_train_client(id, client_loader, global_model, num_local_epochs, lr):
    local_model = copy.deepcopy(global_model)
    local_model = local_model.to(device)
    local_model.train()
    optimizer = torch.optim.SGD(local_model.parameters(), lr=lr, momentum = 0.9)

    for epoch in range(num_local_epochs):
        for (i, (x,y)) in enumerate(client_loader):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            local_out, _ = local_model(x)
      
            loss = criterion(local_out, y)
            loss.backward()
            optimizer.step()
    
    
    return local_model

In [9]:


### 서버에게서 받은 global model.pt


global_model = torch.load('/home/jovyan/Taein/global_model.pt')
client_model = resnet18(10, False)
client_model.load_state_dict(global_model)

client_dataset = torch.load('/home/jovyan/Taein/data/cifar10_split_1.pt')
client_loader = DataLoader(client_dataset, batch_size=32, shuffle=True)


criterion = nn.CrossEntropyLoss()

client_model_1 = avg_train_client(1, client_loader, model, 5, 0.01)

torch.save(client_model_1, '/home/jovyan/Taein/client_model_1.pt')


### 클라이언트는 서버에게 client_model_(i).pt를 준다.

RuntimeError: Error(s) in loading state_dict for ResNet_18:
	Missing key(s) in state_dict: "conv1g.weight", "bng.weight", "bng.bias", "bng.running_mean", "bng.running_var", "layer1.0.conv_1.weight", "layer1.0.bn_1.weight", "layer1.0.bn_1.bias", "layer1.0.bn_1.running_mean", "layer1.0.bn_1.running_var", "layer1.0.conv_2.weight", "layer1.0.bn_2.weight", "layer1.0.bn_2.bias", "layer1.0.bn_2.running_mean", "layer1.0.bn_2.running_var", "layer1.1.conv_1.weight", "layer1.1.bn_1.weight", "layer1.1.bn_1.bias", "layer1.1.bn_1.running_mean", "layer1.1.bn_1.running_var", "layer1.1.conv_2.weight", "layer1.1.bn_2.weight", "layer1.1.bn_2.bias", "layer1.1.bn_2.running_mean", "layer1.1.bn_2.running_var", "layer2.0.conv_1.weight", "layer2.0.bn_1.weight", "layer2.0.bn_1.bias", "layer2.0.bn_1.running_mean", "layer2.0.bn_1.running_var", "layer2.0.conv_2.weight", "layer2.0.bn_2.weight", "layer2.0.bn_2.bias", "layer2.0.bn_2.running_mean", "layer2.0.bn_2.running_var", "layer2.0.downsample.0.weight", "layer2.0

In [ ]:
# 1 라운드 동안.. 접속 완료
# 서버에게 모델을 받음 
# 함수 돌리고
# 모델 저장
# 서버에게 보냄